From raw cnid + description tabel, Transform test data and fit and compare result against true values.

In [27]:
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import nltk
import datetime as dt
import time
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
df_test = pd.read_csv('df_test.csv')

In [29]:
df_test.columns

Index(['Cnid', 'Description', 'idCat', 'yesIT'], dtype='object')

In [30]:
# make cnid index
df_test.index = df_test['Cnid']
df_test.drop(['Cnid'],axis=1,inplace=True)

In [31]:
x_test = df_test.Description
y_test = df_test.yesIT

In [32]:
y_test.head()

Cnid
CN3348181    1
CN3110212    1
CN3442793    1
CN3422590    1
CN3428960    1
Name: yesIT, dtype: int64

In [33]:
type(y_test)

pandas.core.series.Series

In [34]:
print(type(x_test))
x_test.index

<class 'pandas.core.series.Series'>


Index(['CN3348181', 'CN3110212', 'CN3442793', 'CN3422590', 'CN3428960',
       'CN3138062', 'CN3411924', 'CN3393051-A1', 'CN3331279', 'CN3323617-A2',
       ...
       'CN3280807', 'CN3322709', 'CN3115062', 'CN3374183', 'CN3327101',
       'CN3358901', 'CN2974132', 'CN3420277', 'CN2781732', 'CN3364558'],
      dtype='object', name='Cnid', length=22282)

In [35]:
x_test[x_test.isnull()] # no null value

Series([], Name: Description, dtype: object)

In [36]:
# for the next step.
def lem_cont(tokens):
    wnl = nltk.WordNetLemmatizer()
    lemmed = [wnl.lemmatize(t) for t in tokens]
    return lemmed

In [37]:
def removestopword(text):
    stopwremoved = [w for w in text if not w in stopwords.words('english')]
    return stopwremoved

In [38]:
# the Target DataFrame need to have "Description" column. And 'Cnid' index
# Since x_test is not lemmatised, we need to do Step2-4 in CH2.
def Cleanstep234(ser):
    # ser is the series passed to function that contains descriptions.
    # step1: replace 'IT' with 'infotech'
    IT_regex = r'(?<=(\b))IT(?=(\b))'
    ser = ser.str.replace(IT_regex, 'infotech',case=True)
    # make sure no IT is missed.
    print(ser[ser.str.contains('infotech')].count())
    print(ser[ser.str.contains('IT')].count())
    print(ser[ser.str.contains(' IT ')].head())
    
    # step2: before tokenize, replace all non-alpha numeric characters with a space.
    regex2 = r'[^a-zA-Z]'
    ser = ser.str.replace(regex2,' ',case=True)
    
    # step3: fillna
    ser.fillna(value="NoDesc",inplace=True)
    
    # step4: tokenize sentences.
    title_tokenised = ser.str.lower().apply(nltk.word_tokenize)
    #title_tokenised.head()
    
    # step5: Lemmatize each entry but do not return to sentence.
    title_token_lem = title_tokenised.apply(lem_cont)
    
    # step6: remove the stop words
    title_token_lem_nostopword = title_token_lem.apply(removestopword)
    
    # step7: join the tokens back together
    output = title_token_lem_nostopword.apply(lambda w: ' '.join(w))
    
    return output
    print(type(output))

In [39]:
# output the test series to review
x_test_proced = Cleanstep234(x_test)

793
114
Series([], Name: Description, dtype: object)


In [40]:
type(x_test_proced)

pandas.core.series.Series

In [41]:
x_test_proced.index

Index(['CN3348181', 'CN3110212', 'CN3442793', 'CN3422590', 'CN3428960',
       'CN3138062', 'CN3411924', 'CN3393051-A1', 'CN3331279', 'CN3323617-A2',
       ...
       'CN3280807', 'CN3322709', 'CN3115062', 'CN3374183', 'CN3327101',
       'CN3358901', 'CN2974132', 'CN3420277', 'CN2781732', 'CN3364558'],
      dtype='object', name='Cnid', length=22282)

In [43]:
pd.DataFrame(x_test_proced).to_csv("x_test_proced.csv")

## TFIDF transformation

In [44]:
tfidf_vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1,2),vocabulary=)
tfidf = tfidf_vectorizer.transform(x_test_proced)

NotFittedError: TfidfVectorizer - Vocabulary wasn't fitted.

## Load Model and predict test result

In [45]:
clf = joblib.load('RandomForestv2.pkl')

In [ ]:
clf.predict

In [ ]:
# predict with this model
y_test_fit = clf.predict(tfidf)
''' Cannot directly apply this. Need to feed the Description data to the training set to get feature matrix beforehand. 
There could be new words in the new contracts.
Then train original model then apply it to new contract set.
'''

In [24]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [25]:
print(classification_report(y_test_fit,y_test))

NameError: name 'y_train_fit' is not defined